In [1]:
import numpy as np
from collections import defaultdict

## Task 12

In [1]:
def simulate_women(Q, number_women=1000, obser_interval=48):
    n_states = Q.shape[0]
    absorbing_state = n_states - 1
    time_series = []
    
    for _ in range(number_women):
        current_state = 0
        time = 0
        woman_series = [current_state + 1]
        
        while current_state != absorbing_state:
            rate = -Q[current_state, current_state]
            holding_time = np.random.exponential(1/rate) if rate > 0 else float('inf')
            
            if current_state != absorbing_state:
                possible_states = [i for i in range(n_states) if i != current_state]
                probs = Q[current_state, possible_states] / rate
                next_state = np.random.choice(possible_states, p=probs)
            else:
                next_state = current_state
            
            next_obs_time = (time // obser_interval + 1) * obser_interval
            transition_time = time + holding_time
            
            while next_obs_time < transition_time:
                woman_series.append(current_state + 1)
                next_obs_time += obser_interval
            
            time = transition_time
            current_state = next_state
        
        if woman_series[-1] != absorbing_state + 1:
            woman_series.append(absorbing_state + 1)
        
        time_series.append(woman_series)
    
    return time_series

def estimate_Q_from_partial_obs(time_series, n_states=5, obser_interval=48):
    Nij = np.zeros((n_states, n_states))
    Si = np.zeros(n_states)
    
    for series in time_series:
        prev_time = 0
        for i in range(1, len(series)):
            current_time = i * obser_interval
            prev_state = series[i-1] - 1 
            current_state = series[i] - 1
            
            Si[prev_state] += obser_interval
            
            if prev_state != current_state:
                Nij[prev_state, current_state] += 1
    
    Q_est = np.zeros((n_states, n_states))
    for i in range(n_states):
        if Si[i] > 0:
            for j in range(n_states):
                if i != j:
                    Q_est[i, j] = Nij[i, j] / Si[i]

    for i in range(n_states):
        Q_est[i, i] = -np.sum(Q_est[i, :])
    
    return Q_est

In [18]:
Q = np.array([
    [-0.0085, 0.005, 0.0025, 0.0, 0.001],
    [0.0, -0.014, 0.005, 0.004, 0.005],
    [0.0, 0.0, -0.008, 0.003, 0.005],
    [0.0, 0.0, 0.0, -0.009, 0.009],
    [0.0, 0.0, 0.0, 0.0, 0.0]
])

time_series = simulate_women(Q, number_women=1000, obser_interval=48)
Q_estimated = estimate_Q_from_partial_obs(time_series, n_states=5, obser_interval=48)

print("Original Q matrix:")
print(Q)
print("\nEstimated Q matrix:")
print(Q_estimated)

Original Q matrix:
[[-0.0085  0.005   0.0025  0.      0.001 ]
 [ 0.     -0.014   0.005   0.004   0.005 ]
 [ 0.      0.     -0.008   0.003   0.005 ]
 [ 0.      0.      0.     -0.009   0.009 ]
 [ 0.      0.      0.      0.      0.    ]]

Estimated Q matrix:
[[-0.00692367  0.00293564  0.00198017  0.00033926  0.00166861]
 [ 0.         -0.00968567  0.00280976  0.00271838  0.00415753]
 [ 0.          0.         -0.00682759  0.00222022  0.00460737]
 [ 0.          0.          0.         -0.00717487  0.00717487]
 [ 0.          0.          0.          0.         -0.        ]]


In [23]:
print("\nOriginal vs Estimated:")
for i in range(5):
    print(f"State {i+1}:")
    print(f"Original: {Q[i,:]}")
    print(f"Estimated: {Q_estimated[i,:]}")
    print()


Original vs Estimated:
State 1:
Original: [-0.0085  0.005   0.0025  0.      0.001 ]
Estimated: [-0.00692367  0.00293564  0.00198017  0.00033926  0.00166861]

State 2:
Original: [ 0.    -0.014  0.005  0.004  0.005]
Estimated: [ 0.         -0.00968567  0.00280976  0.00271838  0.00415753]

State 3:
Original: [ 0.     0.    -0.008  0.003  0.005]
Estimated: [ 0.          0.         -0.00682759  0.00222022  0.00460737]

State 4:
Original: [ 0.     0.     0.    -0.009  0.009]
Estimated: [ 0.          0.          0.         -0.00717487  0.00717487]

State 5:
Original: [0. 0. 0. 0. 0.]
Estimated: [ 0.  0.  0.  0. -0.]



## Task 13

In [40]:
def simulate_between_observations(Q, observed_series, obser_interval=48, n_simulations=100):
    n_states = Q.shape[0]
    all_transitions = []
    all_sojourns = []
    
    for series in observed_series:
        transitions = defaultdict(int)
        sojourns = defaultdict(float)
        
        for i in range(len(series)-1):
            start_state = series[i] - 1
            end_state = series[i+1] - 1
            time_passed = 0
            current_state = start_state
            
            while time_passed < obser_interval:
                rate = -Q[current_state, current_state]
                holding_time = np.random.exponential(1/rate) if rate > 0 else float('inf')
                
                if current_state != n_states - 1:
                    possible_states = [i for i in range(n_states) if i != current_state]
                    probs = Q[current_state, possible_states] / rate
                    next_state = np.random.choice(possible_states, p=probs)
                else:
                    next_state = current_state
                
                time_remaining = obser_interval - time_passed
                time_in_state = min(holding_time, time_remaining)
                
                sojourns[current_state] += time_in_state
                
                if time_passed + holding_time <= obser_interval:
                    transitions[(current_state, next_state)] += 1
                    current_state = next_state
                else:
                    if current_state == end_state:
                        break
                    else:
                        transitions = defaultdict(int)
                        sojourns = defaultdict(float)
                        time_passed = 0
                        current_state = start_state
                        continue
                
                time_passed += holding_time
        
        all_transitions.append(transitions)
        all_sojourns.append(sojourns)
    
    return all_transitions, all_sojourns

def estimate_Q_from_simulations(all_transitions, all_sojourns, n_states=5):
    Nij = np.zeros((n_states, n_states))
    Si = np.zeros(n_states)
    
    for transitions, sojourns in zip(all_transitions, all_sojourns):
        for (i, j), count in transitions.items():
            Nij[i, j] += count
        for state, time in sojourns.items():
            Si[state] += time
    
    Q_est = np.zeros((n_states, n_states))
    for i in range(n_states):
        if Si[i] > 0:
            for j in range(n_states):
                if i != j:
                    Q_est[i, j] = Nij[i, j] / Si[i]
    
    for i in range(n_states):
        Q_est[i, i] = -np.sum(Q_est[i, :])
    
    return Q_est

def mc_em_algorithm(Q_init, observed_series, obser_interval=48, 
                   max_iter=100, tolerance=1e-3, n_simulations=100):
    Q_current = Q_init.copy()
    prev_Q = Q_current.copy()
    
    for iteration in range(max_iter):
        all_transitions, all_sojourns = simulate_between_observations(
            Q_current, observed_series, obser_interval, n_simulations)
        Q_current = estimate_Q_from_simulations(all_transitions, all_sojourns)
        max_diff = np.max(np.abs(Q_current - prev_Q))
        print(f"Iteration {iteration+1}: Max difference = {max_diff:.6f}")
        if max_diff < tolerance:
            print(f"Converged after {iteration+1} iterations")
            break
            
        prev_Q = Q_current.copy()
    
    return Q_current

In [42]:
Q_true = np.array([
    [-0.0085, 0.005, 0.0025, 0.0, 0.001],
    [0.0, -0.014, 0.005, 0.004, 0.005],
    [0.0, 0.0, -0.008, 0.003, 0.005],
    [0.0, 0.0, 0.0, -0.009, 0.009],
    [0.0, 0.0, 0.0, 0.0, 0.0]
])

print("Simulate observed data:")
observed_series = simulate_women(Q_true, number_women=1000, obser_interval=48)

Q_init = np.array([
    [-0.01, 0.004, 0.003, 0.0, 0.003],
    [0.0, -0.01, 0.005, 0.003, 0.002],
    [0.0, 0.0, -0.01, 0.004, 0.006],
    [0.0, 0.0, 0.0, -0.01, 0.01],
    [0.0, 0.0, 0.0, 0.0, 0.0]
])

Q_estimated = mc_em_algorithm(Q_init, observed_series, obser_interval=48, 
                            tolerance=1e-3, n_simulations=100)


Simulate observed data:
Iteration 1: Max difference = 0.026473
Iteration 2: Max difference = 0.009508
Iteration 3: Max difference = 0.006954
Iteration 4: Max difference = 0.002430
Iteration 5: Max difference = 0.002587
Iteration 6: Max difference = 0.002317
Iteration 7: Max difference = 0.003698
Iteration 8: Max difference = 0.002875
Iteration 9: Max difference = 0.001294
Iteration 10: Max difference = 0.003667
Iteration 11: Max difference = 0.001718
Iteration 12: Max difference = 0.002979
Iteration 13: Max difference = 0.001962
Iteration 14: Max difference = 0.002366
Iteration 15: Max difference = 0.002175
Iteration 16: Max difference = 0.001856
Iteration 17: Max difference = 0.003185
Iteration 18: Max difference = 0.003541
Iteration 19: Max difference = 0.002629
Iteration 20: Max difference = 0.001974
Iteration 21: Max difference = 0.001339
Iteration 22: Max difference = 0.001363
Iteration 23: Max difference = 0.005211
Iteration 24: Max difference = 0.006255
Iteration 25: Max differe

In [43]:
print("\nTrue Q matrix:")
print(Q_true)
print("\nEstimated Q matrix:")
print(Q_estimated)



True Q matrix:
[[-0.0085  0.005   0.0025  0.      0.001 ]
 [ 0.     -0.014   0.005   0.004   0.005 ]
 [ 0.      0.     -0.008   0.003   0.005 ]
 [ 0.      0.      0.     -0.009   0.009 ]
 [ 0.      0.      0.      0.      0.    ]]

Estimated Q matrix:
[[-0.03379006  0.01526603  0.00698142  0.          0.01154261]
 [ 0.         -0.03354616  0.00897853  0.00887987  0.01568776]
 [ 0.          0.         -0.02387504  0.00452137  0.01935367]
 [ 0.          0.          0.         -0.02177552  0.02177552]
 [ 0.          0.          0.          0.         -0.        ]]


In [44]:
relative_error = np.abs((Q_estimated - Q_true) / Q_true)
relative_error[np.isinf(relative_error)] = 0 
print("\nRelative errors:")
print(relative_error)


Relative errors:
[[ 2.97530083  2.05320626  1.7925667          nan 10.54260903]
 [        nan  1.39615451  0.79570638  1.21996668  2.1375529 ]
 [        nan         nan  1.98438022  0.5071235   2.87073425]
 [        nan         nan         nan  1.41950274  1.41950274]
 [        nan         nan         nan         nan         nan]]


C:\Users\Mathias Dyhr\AppData\Local\Temp\ipykernel_2760\805750866.py:1: RuntimeWarning: invalid value encountered in divide
  relative_error = np.abs((Q_estimated - Q_true) / Q_true)
